In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
nfl = pd.read_csv('C:/Users/OYO/Desktop/NFL_Folder/nfl_play.csv',low_memory=False)
nfl.shape

(449371, 255)

In [7]:
data = nfl[['posteam','defteam','qtr','yardline_100','quarter_seconds_remaining','half_seconds_remaining',
                       'posteam_score','defteam_score','no_score_prob','td_prob',
           'touchdown','fourth_down_converted','down','play_type']]
data.head()

,posteam,defteam,qtr,yardline_100,quarter_seconds_remaining,half_seconds_remaining,posteam_score,defteam_score,no_score_prob,td_prob,touchdown,fourth_down_converted,down,play_type
0,PIT,TEN,1,30.0,900.0,1800.0,NaN,NaN,0.001506,0.313676,0.0,0.0,NaN,kickoff
1,PIT,TEN,1,58.0,893.0,1793.0,0.0,0.0,0.000969,0.423011,0.0,0.0,1.0,pass
2,PIT,TEN,1,53.0,856.0,1756.0,0.0,0.0,0.001057,0.421478,0.0,0.0,2.0,run
3,PIT,TEN,1,56.0,815.0,1715.0,0.0,0.0,0.001434,0.318621,0.0,0.0,3.0,pass
4,PIT,TEN,1,56.0,807.0,1707.0,0.0,0.0,0.001861,0.208111,0.0,0.0,4.0,punt


In [8]:
data = data[(data['down'] == 4.0)]
data.shape

(39644, 14)

In [9]:
data.isnull().sum()

posteam                      119
defteam                      119
qtr                            0
yardline_100                 119
quarter_seconds_remaining      0
half_seconds_remaining         2
posteam_score                119
defteam_score                119
no_score_prob                  0
td_prob                        0
touchdown                    119
fourth_down_converted        119
down                           0
play_type                    119
dtype: int64

In [10]:
data.dropna(how='any',inplace=True)
data.shape

(39523, 14)

In [11]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
pos_team = label.fit_transform(data['posteam'])
def_team = label.fit_transform(data['defteam'])
play_type = label.fit_transform(data['play_type'])

data['pos_team'] = pos_team
data['def_team'] = def_team
data['play_type'] = play_type

In [12]:
data.drop(['posteam','defteam'],axis=1,inplace=True)

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[['yardline_100','quarter_seconds_remaining','half_seconds_remaining',
                       'posteam_score','defteam_score']] = scaler.fit_transform(data[['yardline_100','quarter_seconds_remaining','half_seconds_remaining',
                       'posteam_score','defteam_score']])
data.head()

,qtr,yardline_100,quarter_seconds_remaining,half_seconds_remaining,posteam_score,defteam_score,no_score_prob,td_prob,touchdown,fourth_down_converted,down,play_type,pos_team,def_team
4,1,0.210318,1.482393,1.709710,-1.052662,-1.138157,0.001861,0.208111,0.0,0.0,4.0,3,27,33
8,1,1.804706,1.055167,1.495628,-1.052662,-1.138157,0.003023,0.135991,0.0,0.0,4.0,3,33,27
14,1,-0.387578,0.340603,1.137561,-1.052662,-1.138157,0.004547,0.150286,0.0,0.0,4.0,3,27,33
20,1,-1.264491,-0.026131,0.953792,-1.052662,-1.138157,0.001197,0.028760,0.0,0.0,4.0,0,33,27
24,1,1.127091,-0.426892,0.752971,-1.052662,-1.138157,0.015674,0.157581,0.0,0.0,4.0,3,27,33


In [14]:
data['touchdown'] = data['touchdown'].astype('int')
data['fourth_down_converted'] = data['fourth_down_converted'].astype('int')
data.drop('down',axis=1,inplace=True)
data.head()

,qtr,yardline_100,quarter_seconds_remaining,half_seconds_remaining,posteam_score,defteam_score,no_score_prob,td_prob,touchdown,fourth_down_converted,play_type,pos_team,def_team
4,1,0.210318,1.482393,1.709710,-1.052662,-1.138157,0.001861,0.208111,0,0,3,27,33
8,1,1.804706,1.055167,1.495628,-1.052662,-1.138157,0.003023,0.135991,0,0,3,33,27
14,1,-0.387578,0.340603,1.137561,-1.052662,-1.138157,0.004547,0.150286,0,0,3,27,33
20,1,-1.264491,-0.026131,0.953792,-1.052662,-1.138157,0.001197,0.028760,0,0,0,33,27
24,1,1.127091,-0.426892,0.752971,-1.052662,-1.138157,0.015674,0.157581,0,0,3,27,33


In [15]:
X = data.drop('fourth_down_converted',axis=1)
y = data['fourth_down_converted']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)
print(X_train.shape)
print(X_test.shape)

(26480, 12)
(13043, 12)


In [17]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=100, random_state=42)

from sklearn.feature_selection import RFE
selector = RFE(lr_model, 7, step=1)
selector = selector.fit(X, y)
selector.support_

C:\Users\OYO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\OYO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\OYO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\OYO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\OYO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warnin

array([False,  True, False, False,  True,  True,  True,  True,  True,
        True, False, False])

In [18]:
selector.ranking_

array([2, 1, 4, 3, 1, 1, 1, 1, 1, 1, 6, 5])

In [21]:
selector.get_params

<bound method BaseEstimator.get_params of RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=42, solver='warn', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=7, step=1, verbose=0)>

In [22]:
lr_model2 = LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=42, solver='warn', tol=0.0001,
                                 verbose=0, warm_start=False)
lr_model2.fit(X_train, y_train)

y_pred_lr2 = lr_model2.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_lr2)

C:\Users\OYO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9514682205014184